# Лабораторная работа №3 - Управление манипуляционным роботом
### Абрамов Максим Витальевич R4134с
### Цель:
> Решить задачу движения по траектории для
 многозвенного манипуляционного робота с помощью ПД-регулятора.
### Задачи:
 > 1. По выбранному варианту кинематики робота(см. таблицу 2.1)
 загрузить модель манипулятора из программного пакета.
 2. Заполнить все параметры модели робота согласно лабораторной работе №1.
 3. Задать желаемую траекторию движения манипулятора согласно лабораторной работе №2.
 4. Создать модель системы управления роботом, реализующую
 регулирование на основе обратной модели динамики. В работе рекомендуется синтезировать ПД-регулятор (допустимо использование других регуляторов).
 5. Настроить коэффициенты регулятора, минимизирующие
 ошибку обобщенных координат звеньев.
 6. Вывести графики изменения положения, скорости и момента
 для каждого из звеньев робота при движении вдоль траектории.
 7. Добавить полезную нагрузку и повторить пункты № 5-6.
 8. Составить отчёт в формате `.ipynb` (см. приложение В).

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from roboticstoolbox import models
import roboticstoolbox as rtb

 ### 1. По выбранному варианту кинематики робота(см. таблицу 2.1) загрузить модель манипулятора из программного пакета.

In [18]:
robot = models.DH.UR5()

### 2. Заполнить все параметры модели робота согласно лабораторной работе №1.

In [19]:
# Массы звеньев (кг)
masses = [3.7, 6.8, 4.8, 0.8, 0.8, 0.8]

# Центр масс каждого звена (в локальной системе координат)
centers_of_mass = [
    [0, 0, -0.08],   # Звено 1
    [0, -0.12, 0],   # Звено 2
    [0, 0, -0.12],   # Звено 3
    [0, 0, 0],       # Звено 4
    [0, 0, 0],       # Звено 5
    [0, 0, 0]        # Звено 6
]

# Тензоры инерции (Lxx, Lyy, Lzz, Lxy, Lyz, Lxz)
inertias = [
    [0.0, 0.09, 0.09, 0, 0, 0],         # Звено 1
    [0.23, 0.2, 0.03, 0, 0, 0],         # Звено 2
    [0.02, 0.02, 0.03, 0, 0, 0],        # Звено 3
    [0.001, 0.001, 0.001, 0, 0, 0],     # Звено 4
    [0.001, 0.001, 0.001, 0, 0, 0],     # Звено 5
    [0.001, 0.001, 0.001, 0, 0, 0]      # Звено 6
]

# Момент инерции двигателей (примерно)
Jm = [0.0001, 0.0004, 0.0001, 0.00001, 0.00001, 0.00001]

# Коэффициенты вязкого трения
B = [0.1, 0.1, 0.05, 0.01, 0.01, 0.01]

# Коэффициенты кулоновского трения (±)
Tc = [0.3, 0.5, 0.3, 0.1, 0.1, 0.1]

# Передаточные числа редукторов
G = [100, 100, 100, 50, 50, 50]

# Ограничения по обобщенным координатам (радианы)
qlim = [
    [-np.pi, np.pi],
    [-np.pi, np.pi],
    [-np.pi, np.pi],
    [-np.pi, np.pi],
    [-np.pi, np.pi],
    [-np.pi, np.pi]
]

# Присвоение динамических параметров каждому звену
for i in range(6):
    robot.links[i].m = masses[i]
    robot.links[i].r = centers_of_mass[i]
    robot.links[i].I = inertias[i]
    robot.links[i].Jm = Jm[i]
    robot.links[i].B = B[i]
    robot.links[i].Tc = Tc[i]
    robot.links[i].G = G[i]
    robot.links[i].qlim = qlim[i]

### 3. Задать желаемую траекторию движения манипулятора согласно лабораторной работе №2.

In [20]:
# Произвольные начальная и конечная конфигурации
q_start = np.array([0, -np.pi/2, np.pi/2, 0, np.pi/2, 0])
q_end = np.array([np.pi/2, -np.pi/3, np.pi/3, 0, np.pi/2, 0])


In [21]:
# Задание временного массива
dt = 0.1
mT = 1
steps = int(mT / dt)
time_array = np.linspace(0, mT, steps)

tr_1 = rtb.mtraj(rtb.trapezoidal, q_start, q_end, time_array)

q_d_array = tr_1.q
qd_d_array = tr_1.qd
qdd_d_array = tr_1.qdd

### 4. Создать модель системы управления роботом, реализующую регулирование на основе обратной модели динамики. В работе рекомендуется синтезировать ПД-регулятор (допустимо использование других регуляторов).

In [22]:
def regfunc(robot, t, q_act, qd_act):
    global Kp, Kd, tau_array
    idx = min(int(t / dt), len(q_d_array) - 1)

    q_d = q_d_array[idx]
    qd_d = qd_d_array[idx]
    qdd_d = qdd_d_array[idx]
    
    e = q_d - q_act
    ed = qd_d - qd_act

    M = robot.inertia(q_act)
    C = robot.coriolis(q_act, qd_act) @ qd_d
    G = robot.gravload(q_act)

    tau = M @ qdd_d + C + G + Kp @ e + Kd @ ed
    tau_array.append(tau)
    return tau


 ### 5. Настроить коэффициенты регулятора, минимизирующие ошибку обобщенных координат звеньев.

In [23]:
Kp = np.diag([470000, 497000, 19000, 37000, 39700, 19000])
Kd = np.diag([30000, 10000, 800, 50, 2, 2])

In [24]:
tau_array = []
tg = robot.fdyn(T=mT, 
                q0=q_start, 
                Q=regfunc, 
                qd0=np.zeros(6), 
                dt=dt, 
                solver="RK45", 
                solver_args={"atol": 1e-3, "rtol": 1e-2}, 
                progress=True)


### 6. Вывести графики изменения положения, скорости и момента для каждого из звеньев робота при движении вдоль траектории.

In [25]:
def plot_pos_and_v(q, dq, colors, labels):
    global t_vis
    # Графики положения
    plt.figure(figsize=(12, 8))
    for i in range(6):
        plt.subplot(2, 3, i+1)
        for j in range(len(q)):
            plt.plot(t_vis, np.degrees(q[j][:, i]), colors[j], label=labels[j])   
        plt.xlabel('Время (с)')
        plt.ylabel(f'q_{i+1} (град)')
        plt.title(f'Соединение {i+1}')
        plt.legend()
        plt.grid(True)
    plt.suptitle('Положения соединений')
    plt.tight_layout()
    plt.show()
    
    # Графики скоростей
    plt.figure(figsize=(12, 8))
    for i in range(6):
        plt.subplot(2, 3, i+1)
        for j in range(len(dq)):
            plt.plot(t_vis, np.degrees(dq[j][:, i]), colors[j], label=labels[j])   
        plt.xlabel('Время (с)')
        plt.ylabel(f'dq_{i+1}/dt (град/с)')
        plt.title(f'Соединение {i+1}')
        plt.legend()
        plt.grid(True)
    plt.suptitle('Скорости соединений')
    plt.tight_layout()
    plt.show()

def plot_momentums(tau, colors, labels):
    global t_vis
    # Графики моментов
    plt.figure(figsize=(12, 8))
    for i in range(6):
        plt.subplot(2, 3, i+1)
        for j in range(len(tau)):
            plt.plot(t_vis, np.degrees(tau[j][:, i]), colors[j], label=labels[j])   
        plt.xlabel('Время (с)')
        plt.ylabel(f'τ_{i+1} (Н·м)')
        plt.title(f'Соединение {i+1}')
        plt.grid(True)
        plt.legend()
    plt.suptitle('Управляющие моменты')
    plt.tight_layout()
    plt.show()

In [26]:
q_sim = np.array(tg.q)

min_len = min(len(tg.t), len(q_d_array))
t_vis = np.array(tg.t)[:min_len]
q_sim_vis = np.array(tg.q)[:min_len]
q_des_vis = q_d_array[:min_len]
qd_sim_vis = np.array(tg.qd)[:min_len]
qd_des_vis = qd_d_array[:min_len]
tau = np.array(tau_array)[:min_len]

plot_pos_and_v([q_des_vis, q_sim_vis], [qd_des_vis, qd_sim_vis], ['r--', 'g'], labels=['Желаемое', 'Реальное (без нагрузки)'])
plot_momentums([tau_traj], ['g'], ['Реальное (без нагрузки)'])


fig, axs = plt.subplots(3, 2, figsize=(14, 10))
axs = axs.flatten()

for i in range(6):
    axs[i].plot(t_vis, q_des_vis[:, i], '--', label='Желаемая траектория')
    axs[i].plot(t_vis, q_sim_vis[:, i], label='Реальная траектория')
    axs[i].set_title(f'Звено {i+1}')
    axs[i].set_xlabel("Время (с)")
    axs[i].set_ylabel("Положение (рад)")
    axs[i].grid(True)
    axs[i].legend()

plt.suptitle("Сравнение желаемой и реальной траекторий UR5 по каждому звену", fontsize=16)
plt.tight_layout()
plt.show()

AttributeError: 'fdyn' object has no attribute 'tau'

 ### 7. Добавить полезную нагрузку и повторить пункты № 5-6.

 ## Выводы о проделанной работе:
 > В рамках лабораторной работы была решена задача управления многозвенным манипуляционным роботом с использованием ПД-регулятора. На основе выбранной кинематической структуры была загружена модель манипулятора `UR5`, все параметры которой были приведены в соответствие с результатами первой лабораторной работы. Заданная траектория движения, спланированная во второй лабораторной работе, использовалась как эталон для построения управляющих воздействий.
>
> Для управления движением манипулятора был синтезирован ПД-регулятор. Путем подбора коэффициентов пропорциональной и дифференциальной составляющих удалось достичь устойчивого движения по заданной траектории с минимальной ошибкой в обобщённых координатах.
> 
> Дополнительно была рассмотрена задача управления при наличии полезной нагрузки. Это позволило проанализировать влияние дополнительной массы на динамику системы и адаптивность регулятора. Результаты показали, что корректно настроенный ПД-регулятор способен обеспечить приемлемое качество управления даже при изменении динамических свойств робота.

